In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import xarray as xr
from ftplib import FTP
import urllib
import pandas as pd
import scipy.interpolate as interp

In [ ]:
from distributed import Client

In [ ]:
client = Client()

In [ ]:
client

# Real Iceberg

In [ ]:
year = 2015 # 2002 through 2015 available
iip_url_base = 'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/' 
iip_filename = 'IIP_{}IcebergSeason.csv'.format(year)
iip_url = iip_url_base + iip_filename

In [ ]:
r = urllib.request.urlretrieve(iip_url)
df = pd.read_csv(r[0])

In [ ]:
n = df.BERG_NUMBER.mode()[0]

In [ ]:
real_berg = df.loc[df['BERG_NUMBER'] == n]

In [ ]:
real_berg_track = real_berg.loc[6318:6333]

In [ ]:
some_dates = real_berg_track.DATE + ' ' + real_berg_track.TIME.astype(str)
pd.to_datetime(some_dates)

In [ ]:
real_berg_track['time'] = pd.to_datetime(some_dates)

In [ ]:
real_berg_track

# Input Fields

## Ocean (GLBv0.08)

- 3 hr
- 0.08 degree

In [ ]:
glb_url = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_56.3'

In [ ]:
glb_ds = xr.open_dataset(glb_url, decode_times=False)

In [ ]:
# June 1, 00:00 = 135120., July 1, 00:00 = 135840.
glb_ds = glb_ds.sel(depth=0.0, lat = slice(45., 51.), lon = slice(-53., -47.), time=slice(135120., 135840.))

In [ ]:
water_u_data = glb_ds.water_u
water_v_data = glb_ds.water_v
water_temp_data = glb_ds.water_temp

In [ ]:
water_u_data.time

In [ ]:
glb_times = np.asarray(water_u_data.time)
glb_lats = np.asarray(water_u_data.lat)
glb_lons = np.asarray(water_u_data.lon)

In [ ]:
glb_times = glb_times - 135120

#### Warning -- the box below takes a few minutes to run

In [ ]:
water_u = np.asarray(water_u_data)
water_v = np.asarray(water_v_data)
water_temp = np.asarray(water_temp_data)

In [ ]:
water_u_interp = interp.RegularGridInterpolator((glb_times, glb_lats, glb_lons), water_u)
water_v_interp = interp.RegularGridInterpolator((glb_times, glb_lats, glb_lons), water_v)
water_temp_interp = interp.RegularGridInterpolator((glb_times, glb_lats, glb_lons), water_temp)

## Atmosphere (NAVGEM)

- 6 hr
- 0.5 degree

In [ ]:
navgem_url = 'http://coastwatch.pfeg.noaa.gov/erddap/griddap/erdNavgem05D10mWind_LonPM180'

In [ ]:
navgem_ds = xr.open_dataset(navgem_url)

In [ ]:
navgem_ds = navgem_ds.sel(time=slice('2015-06-01','2015-07-01'), latitude=slice(45,51), longitude=slice(-53,-47))

In [ ]:
wind_u = navgem_ds.wnd_ucmp_height_above_ground
wind_v = navgem_ds.wnd_vcmp_height_above_ground

In [ ]:
navgem_times = np.linspace(0, 120*6 - 6, 120)
navgem_lats = np.asarray(wind_u.latitude)
navgem_lons = np.asarray(wind_u.longitude)

In [ ]:
wind_u = np.asarray(wind_u[:,0,:,:])
wind_v = np.asarray(wind_v[:,0,:,:])

In [ ]:
wind_u_interp = interp.RegularGridInterpolator((navgem_times, navgem_lats, navgem_lons), wind_u)
wind_v_interp = interp.RegularGridInterpolator((navgem_times, navgem_lats, navgem_lons), wind_v)

# Input Analysis

In [ ]:
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-40, -60, 40, 60], ccrs.PlateCarree())
#ax.stock_img()
ax.coastlines('50m')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
#gl.xlabels_top = False
#gl.ylabels_left = False
#gl.xlines = False
#gl.xlocator = mticker.FixedLocator([-180, -45, 0, 45, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}
ax.plot(real_berg_track['LONGITUDE'], real_berg_track['LATITUDE'])
plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
from IPython.display import HTML

In [ ]:
water_mag = np.sqrt(water_u**2 + water_v**2)

In [ ]:
(real_berg_track.LATITUDE.values[0], real_berg_track.LONGITUDE.values[0])

In [ ]:
np.datetime64(npdt, 's')

In [ ]:
rbt_days_arr = pd.DatetimeIndex(real_berg_track.time).day

In [ ]:
rbt_hours_arr = pd.DatetimeIndex(real_berg_track.time).hour

In [ ]:
rbt_minutes_arr = pd.DatetimeIndex(real_berg_track.time).minute

In [ ]:
def rbt_hours_since(days_arr, hours_arr, mins_arr):
    days_arr = days_arr - days_arr[0]  # must be same month
    rbt_hours_since = np.empty(days_arr.size)
    for i in range(rbt_hours_since.size):
        rbt_hours_since[i] = days_arr[i]*24 + hours_arr[i] + mins_arr[i]/60
    return rbt_hours_since

In [ ]:
rbt_hours_since_arr

In [ ]:
rbt_hours_since_arr = rbt_hours_since(rbt_days_arr, rbt_hours_arr, rbt_minutes_arr)

In [ ]:
glb_times[50] - rbt_hours_since_arr

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-54, -44, 44, 54], ccrs.PlateCarree())
#ax.stock_img()
ax.coastlines('50m')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

#ax.plot(real_berg_track['LONGITUDE'], real_berg_track['LATITUDE'])

line, = plt.plot(real_berg_track.LONGITUDE.values[0], real_berg_track.LATITUDE.values[0], 'o')
im = plt.imshow(water_mag[0,:,:], extent=[glb_lons[0], glb_lons[-1], glb_lats[0], glb_lats[-1]],
                origin = 'lower', vmin = -1, vmax = 1)
plt.colorbar()
title = plt.title('')


def animate(i):

    im.set_data(water_mag[i,:,:])
    
    time = glb_times[i]
    days = time//24
    hours = time%24
    diff = abs(time - rbt_hours_since_arr)

    if any(diff) <= 3:
        k = np.where(diff <= 3)[0]
        line.set_data(real_berg_track.LONGITUDE.values[k], real_berg_track.LATITUDE.values[k])
        ax.plot(real_berg_track.LONGITUDE.values[k], real_berg_track.LATITUDE.values[k], marker='o')

        
    title.set_text('time: {:.0f} days {:.0f} hours'.format(days, hours))
    
    return im, title, line

anim = FuncAnimation(fig, animate, frames=100)

In [ ]:
HTML(anim.to_html5_video())

# Model

In [ ]:
import scipy.io as sio
import numpy.matlib
import cmath

In [ ]:
# Constants
R = 6378*1e3
om = 7.2921e-5
rhow = 1027
rhoa = 1.2
rhoi = 850
drho = rhow - rhoi
Cw = 0.9
Ca = 1.3
gam = np.sqrt(rhoa*drho/rhow/rhoi*(Ca/Cw))
sst0 = -4
Cs1 = 1.5; Cs2 = 0.5; Cs3 = 0.1
CMv1 = 7.62e-3; CMv2 = 1.29e-3; CMe1 = 0.5
CMb1 = 0.58; CMb2 = 0.8; CMb3 = 0.2

In [ ]:
def iceDEF(t,x,y,l,w,h):

    #print('\ntimestep: {}\n'.format(t))
    

    # Extract values from input fields
    
    vau = wind_u_interp([t, y, x])[0]
    vav = wind_v_interp([t, y, x])[0]  
    vwu = water_u_interp([t, y, x])[0] 
    vwv = water_v_interp([t, y, x])[0]
    sst = water_temp_interp([t, y, x])[0]

    #print('vau = ', vau)
    #print('vav = ', vav)
    #print('vwu = ', vwu)
    #print('vwv = ', vwv)
    #print('sst = ', sst)
    

    # Drifting

    S = np.pi*((l*w)/(l+w))
    ff = 2*om*np.sin((np.abs(y)*np.pi)/180)
    lam = np.sqrt(2)*Cw*(gam*np.sqrt(vau**2 + vav**2))/(ff*S)
    #print('S = {0:.15f}'.format(S))
    #print('ff = {0:.15f}'.format(ff))
    #print('lam = {0:.15f}'.format(lam))
    
    if lam < 0.1:
        #print('Taylor approx used for alpha')
        alpha = lam*(lam**4*(lam**4*(lam**4*(-0.0386699020961393*lam**4 + \
            0.055242717280199) - 0.0883883476483184) + \
            0.176776695296637) - 0.707106781186548)
    else:
        alpha = np.multiply(np.divide(np.sqrt(2),np.power(lam, 3)),(1-np.sqrt(1+np.power(lam,4))))
        
    if lam < 0.6:
        #print('Taylor approx used for beta')
        beta = lam**3*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*\
            (0.0153268598203613*lam**4 - 0.0151656272365985) + \
            0.0180267866272764) + 0.0219176256311202) - \
            0.0274446790511418) + 0.0357675015202851) - \
            0.0493731785691779) + 0.0745776683282687) - \
            0.132582521472478) + 0.353553390593274)
    else:
        beta = np.real(np.multiply(np.divide(1.,np.power(lam,3.)),cmath.sqrt(np.multiply((4.+np.power(lam,4.)), \
            cmath.sqrt(1.+np.power(lam,4.)))-3.*np.power(lam,4.)-4.)))

    #print('alpha = {0:.15f}'.format(alpha))
    #print('beta = {}'.format(beta))

    viu = vwu + gam*(-alpha*vav + beta*vau)
    viv = vwv + gam*(alpha*vau + beta*vav)

    #print('viu = {0:.15f}'.format(viu))
    #print('viv = {0:.15f}'.format(viv))

    y_new = y + (viv*dt)*(180/(np.pi*R))
    x_new = x + (viu*dt)/(np.cos((((y + y_new)/2)*np.pi)/180))*(180/(np.pi*R))

    #print('x_new = {0:.15f}'.format(x_new))
    #print('y_new = {0:.15f}'.format(y_new))
    
    
    # Melting

    Me = CMe1*(Cs1*np.sqrt(vau**2 + vav**2)**Cs2 + Cs3*np.sqrt(vau**2 + vav**2))
    Mv = CMv1*sst + CMv2*sst**2
    Mb = CMb1*np.power(np.sqrt(np.square(viu-vwu)+np.square(viv-vwv)),CMb2)*(sst - sst0)/l**CMb3

    #print('Me = {0:.15f}'.format(Me))
    #print('Mv = {0:.15f}'.format(Mv))
    #print('Mb = {0:.15f}'.format(Mb))

    l_new = l - (Mv + Me)*(dt/(24*3600))  # convert dt from secs to days
    w_new = w - (Mv + Me)*(dt/(24*3600))
    h_new = h - Mb*(dt/(24*3600))

    if w_new < 0.85*h_new:
        # Rollover
        print('rollover')
        w_new, h_new = h_new, w_new

    if w_new > l_new:
        # Ensure l is greater than w
        print('swap l and w')
        w_new, l_new = l_new, w_new

    #print('l_new = {0:.15f}'.format(l_new))
    #print('w_new = {0:.15f}'.format(w_new))
    #print('h_new = {0:.15f}'.format(h_new))

    return x_new, y_new, l_new, w_new, h_new    


In [ ]:
# Timesteps
t0 = 360 + 11  # initial hour
t_inc = 1  # hours
num_days = 14
tn = t_inc*num_days*24  # total number of timesteps
dt = t_inc*3600  # model timestep in seconds
tf = t0 + tn*dt  # final timestep in seconds  
t_all = np.arange(t0, t0+tn, t_inc)

In [ ]:
rbt_hours_since_arr

In [ ]:
# Iceberg Initial Location
x0, y0 = -50.14, 48.49   # lon, lat

# Run number
run_num = 0
num_runs = 1

for run_num in range(num_runs):
    
    # Iceberg Initial Size
    # Large berg: 
        # Height: 46 metres to 75 metres above surface 
        # Length or width: 121 metres to 200 metres
    l0 = 121
    w0 = 121
    h0 = 46*10
    l0 = l0 + l0*0.1*run_num
    w0 = w0 + w0*0.1*run_num
    h0 = h0 + h0*0.1*run_num
    l, w, h = l0, w0, h0
    
    x, y = x0, y0
    t = t0
    iceberg = np.array([[t0],[x0],[y0],[l0],[w0],[h0]])

    print('Run number: {}, h = {}, l = {}, w = {}'.format(run_num, h0, l0, w0))
    
    while t < max(t_all):

        x_new, y_new, l_new, w_new, h_new = iceDEF(t, x, y, l, w, h)


        if x_new > -47.1 or x_new < -53 or y_new > 51 or y_new < 45:
            # Iceberg out-of-bounds
            #print(x_new)
            #print(y_new)
            print('out-of-bounds')
            break


        if l_new <= 0 or w_new <= 0 or h_new <= 0:
            # Iceberg melted
            print('melted')
            break

        else:
            x, y, l, w, h = x_new, y_new, l_new, w_new, h_new
            t += t_inc
            iceberg_new = np.array([[t],[x],[y],[l],[w],[h]])
            iceberg = np.column_stack((iceberg, iceberg_new))

    plt.plot(real_berg_track['LONGITUDE'], real_berg_track['LATITUDE'], label='observed', marker='o')
    plt.plot(iceberg[1,:],iceberg[2,:], label='computed')
    
    for real_berg_hour in rbt_hours_since_arr: 
        diff = abs((iceberg[0,:]-360) - real_berg_hour)
        k = np.where(diff < (t_inc/2))[0]
        plt.plot(iceberg[1,k],iceberg[2,k],marker='D')
        if len(k) >= 1:
            plt.annotate('{0:.1f}'.format(real_berg_hour), xy=(iceberg[1,k][0],iceberg[2,k][0]))
    #plt.legend()

    if run_num == 0:
        output_dict = {
                        't_arr{}'.format(run_num): iceberg[0,:],
                        'x_arr{}'.format(run_num): iceberg[1,:],
                        'y_arr{}'.format(run_num): iceberg[2,:],
                        'l_arr{}'.format(run_num): iceberg[3,:],
                        'w_arr{}'.format(run_num): iceberg[4,:],
                        'h_arr{}'.format(run_num): iceberg[5,:]
                    }
    else:
        output_dict['t_arr{}'.format(run_num)] = iceberg[0,:]
        output_dict['x_arr{}'.format(run_num)] = iceberg[1,:]
        output_dict['y_arr{}'.format(run_num)] = iceberg[2,:]
        output_dict['l_arr{}'.format(run_num)] = iceberg[3,:]
        output_dict['w_arr{}'.format(run_num)] = iceberg[4,:]
        output_dict['h_arr{}'.format(run_num)] = iceberg[5,:]
